In [5]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
import tkinter as tk
import time

# Fonction pour initialiser la grille avec deux tuiles au départ
def initialize_game():
    board = [[0] * 4 for _ in range(4)]
    add_new_tile(board)
    add_new_tile(board)
    return board

# Fonction pour ajouter une nouvelle tuile (2 ou 4) à une position vide
def add_new_tile(board):
    empty_tiles = [(r, c) for r in range(4) for c in range(4) if board[r][c] == 0]
    if empty_tiles:
        r, c = random.choice(empty_tiles)
        board[r][c] = 2 if random.random() < 0.9 else 4

# Fonction pour fusionner une ligne ou une colonne de tuiles
def merge(line):
    non_zero = [num for num in line if num != 0]  # Supprimer les zéros
    merged = []
    score = 0  # Initialiser le score pour cette ligne
    skip = False

    for i in range(len(non_zero)):
        if skip:
            skip = False
            continue
        if i + 1 < len(non_zero) and non_zero[i] == non_zero[i + 1]:
            merged.append(non_zero[i] * 2)
            score += non_zero[i] * 2  # Ajouter la valeur fusionnée au score
            skip = True
        else:
            merged.append(non_zero[i])

    merged.extend([0] * (len(line) - len(merged)))  # Compléter avec des zéros à droite
    return merged, score  # Retourner la ligne fusionnée et le score

# Fonction pour déplacer et fusionner les tuiles à gauche
def move_left(board):
    score = 0  # Initialiser le score pour ce mouvement
    for i in range(4):
        new_row, gained_score = merge(board[i])  # Fusionner les tuiles et obtenir le score
        board[i] = new_row  # Mettre à jour la ligne
        score += gained_score  # Ajouter le score obtenu lors de la fusion
    return board, score  # Retourner le plateau mis à jour et le score total

# Fonction pour déplacer et fusionner les tuiles à droite
def move_right(board):
    score = 0
    for i in range(4):
        new_row, gained_score = merge(board[i][::-1])  # Fusionner après avoir inversé la ligne
        board[i] = new_row[::-1]  # Réinverser après la fusion
        score += gained_score
    return board, score

# Fonction pour déplacer et fusionner les tuiles vers le haut
def move_up(board):
    score = 0
    for col in range(4):
        column = [board[row][col] for row in range(4)]
        new_column, gained_score = merge(column)
        score += gained_score
        for row in range(4):
            board[row][col] = new_column[row]
    return board, score

# Fonction pour déplacer et fusionner les tuiles vers le bas
def move_down(board):
    score = 0
    for col in range(4):
        column = [board[row][col] for row in range(4)]
        new_column, gained_score = merge(column[::-1])  # Fusionner après avoir inversé la colonne
        score += gained_score
        for row in range(4):
            board[row][col] = new_column[::-1][row]  # Réinverser après la fusion
    return board, score

# Classe de l'environnement du jeu 2048
class Game2048Env:
    def __init__(self):
        self.reset()

    def reset(self):
        self.board = initialize_game()
        self.total_score = 0  # Ajoute une variable pour suivre le score total du jeu
        return np.array(self.board)

    def step(self, action):
        prev_board = np.copy(self.board)
        score = 0  # Initialiser le score

        if action == 0:
            self.board, score = move_up(self.board)
        elif action == 1:
            self.board, score = move_down(self.board)
        elif action == 2:
            self.board, score = move_left(self.board)
        elif action == 3:
            self.board, score = move_right(self.board)

        if not np.array_equal(prev_board, self.board):
            add_new_tile(self.board)

        # Ajoute le score des fusions réalisées dans ce mouvement au score total
        self.total_score += score

        done = check_win(self.board) or check_game_over(self.board)

        return np.array(self.board), score, done  # Retourner le plateau et le score total pour ce mouvement

# Fonction pour vérifier si le joueur a gagné
def check_win(board):
    return any(2048 in row for row in board)

# Fonction pour vérifier s'il reste des déplacements possibles
def check_game_over(board):
    if any(0 in row for row in board):
        return False
    for row in board:
        for i in range(3):
            if row[i] == row[i + 1]:
                return False
    for col in range(4):
        for i in range(3):
            if board[i][col] == board[i + 1][col]:
                return False
    return True

# Classe DQNAgent pour l'entraînement par renforcement
class DQNAgent:
    def __init__(self, input_shape, num_actions):
        self.num_actions = num_actions
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount rate
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001  # Learning rate

        # Création du modèle de Q-Network et du Target Network
        self.model = self.create_model(input_shape, num_actions)
        self.target_model = self.create_model(input_shape, num_actions)

    def create_model(self, input_shape, num_actions):
        model = tf.keras.Sequential([
            layers.Input(shape=input_shape),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(num_actions, activation='linear')  # Output: Q-values for each action
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def update_target_model(self):
        # Met à jour le modèle cible avec les poids du modèle principal
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        # Sauvegarde la transition dans la mémoire
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Politique ε-greedy : explore avec probabilité epsilon, sinon exploite
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.num_actions)
        q_values = self.model.predict(state, verbose=0)  # Prédit les Q-values pour chaque action
        return np.argmax(q_values[0])  # Choisit l'action avec la Q-value la plus élevée

    def replay(self):
        if len(self.memory) < 64:  # Attends que la mémoire ait assez d'expériences
            return
        minibatch = random.sample(self.memory, 64)  # Sélectionne un échantillon de la mémoire
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state, verbose=0)
            if done:
                target[0][action] = reward  # Récompense si l'épisode est terminé
            else:
                t = self.target_model.predict(next_state, verbose=0)
                target[0][action] = reward + self.gamma * np.amax(t[0])  # Q-value cible

            self.model.fit(state, target, epochs=1, verbose=0)

        # Réduit epsilon (exploration) au fil du temps
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Interface graphique tkinter pour visualiser l'entraînement
class Game2048EnvGUI(Game2048Env):
    def __init__(self, root):
        super().__init__()
        self.root = root
        self.root.title("2048 Training")
        self.canvas = tk.Canvas(root, width=400, height=400)
        self.canvas.pack()
        self.tiles = [[self.canvas.create_text(100 * j + 50, 100 * i + 50, text='', font=("Helvetica", 30))
                       for j in range(4)] for i in range(4)]

    def update_gui(self):
        for i in range(4):
            for j in range(4):
                value = self.board[i][j]
                self.canvas.itemconfig(self.tiles[i][j], text=str(value) if value != 0 else '')
        self.root.update()

if __name__ == "__main__":
    root = tk.Tk()
    env_gui = Game2048EnvGUI(root)
    agent = DQNAgent(input_shape=(4, 4), num_actions=4)
    
    episodes = 10  # Réduit pour visualiser rapidement
    for e in range(episodes):
        state = env_gui.reset().reshape(1, 4, 4)
        done = False
        total_reward = 0
        total_score = 0  # Pour suivre le score total du jeu

        while not done:
            action = agent.act(state)
            next_state, reward, done = env_gui.step(action)
            next_state = next_state.reshape(1, 4, 4)

            total_reward += reward
            total_score = env_gui.total_score  # Suivre le score total
            
            agent.remember(state, action, reward, next_state, done)
            state = next_state

            env_gui.update_gui()  # Met à jour l'interface graphique avec le nouvel état du jeu
            time.sleep(0.5)  # Délai pour visualiser les actions

            if done:
                print(f"Episode: {e+1}/{episodes}, Total Reward: {total_reward}, Total Score: {total_score}")
                agent.update_target_model()
                
        agent.replay()

    root.mainloop()

Episode: 1/10, Total Reward: 2520, Total Score: 2520
Episode: 2/10, Total Reward: 1188, Total Score: 1188
Episode: 3/10, Total Reward: 584, Total Score: 584
Episode: 4/10, Total Reward: 632, Total Score: 632
Episode: 5/10, Total Reward: 772, Total Score: 772
Episode: 6/10, Total Reward: 1108, Total Score: 1108
Episode: 7/10, Total Reward: 1296, Total Score: 1296
Episode: 8/10, Total Reward: 1312, Total Score: 1312
Episode: 9/10, Total Reward: 1508, Total Score: 1508
Episode: 10/10, Total Reward: 580, Total Score: 580
